In [2]:
from protein_folding.protein_folding_problem import ProteinFoldingProblem
from protein_folding.peptide.peptide import Peptide
from protein_folding.interactions.random_interaction import RandomInteraction
from protein_folding.interactions.miyazawa_jernigan_interaction import MiyazawaJerniganInteraction
from protein_folding.penalty_parameters import PenaltyParameters
from qiskit.utils import algorithm_globals, QuantumInstance
algorithm_globals.random_seed = 23

In [33]:
# qiskit_research tutorial chain
# main_chain = "APRLRFY" 
# main_chain = "APRLR" 
# fractal analytics Alzheimer's enzyme related chain
main_chain = "YPYFIP"
main_chain_len = len(main_chain)
print("main_chain_len", main_chain_len)

main_chain_len 6


In [34]:
side_chains = [""] * main_chain_len
random_interaction = RandomInteraction()
mj_interaction = MiyazawaJerniganInteraction()
penalty_back = 10
penalty_chiral = 10
penalty_1 = 10
penalty_terms = PenaltyParameters(penalty_chiral, penalty_back, penalty_1)
peptide = Peptide(main_chain, side_chains)
protein_folding_problem = ProteinFoldingProblem(peptide, mj_interaction, penalty_terms)
qubit_op = protein_folding_problem.qubit_op()
print(qubit_op)

929.4905 * IIIIII
+ 300.0 * IIIZII
- 97.5 * IIIIZZ
+ 97.5 * IIIZZZ
- 100.0 * IZIZII
- 100.0 * IIZIZI
- 100.0 * IZZZZI
+ 202.5 * IIIZZI
- 310.0 * IZIIII
- 207.5 * IZZIII
+ 205.0 * IIIIIZ
+ 102.5 * IIZIIZ
- 102.5 * IZZIIZ
- 924.4905 * ZIIIII
- 302.5 * ZIIZII
- 202.5 * ZIIZZI
+ 310.0 * ZZIIII
+ 207.5 * ZZZIII
+ 102.5 * ZZIZII
+ 102.5 * ZIZIZI
+ 102.5 * ZZZZZI
- 205.0 * ZIIIIZ
+ 100.0 * ZIIIZZ
- 100.0 * ZIIZZZ
- 102.5 * ZIZIIZ
+ 102.5 * ZZZIIZ
- 2.5 * IIIIZI
+ 2.5 * IIZIII
+ 2.5 * ZIIIZI
- 2.5 * ZIZIII


In [37]:
print(qubit_op.num_qubits, 2**qubit_op.num_qubits)

6 64


In [38]:
qubit_op_matrix = qubit_op.to_matrix()
print(type(qubit_op_matrix))
print(qubit_op_matrix.shape)
#qubit_op_matrix

<class 'numpy.ndarray'>
(64, 64)


In [39]:
import scipy.linalg as la
qubit_op_unitary = la.expm(qubit_op_matrix)
qubit_op_unitary

array([[2.20264658e+04+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, ...,
        0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j],
       [0.00000000e+00+0.j, 2.20264658e+04+0.j, 0.00000000e+00+0.j, ...,
        0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j],
       [0.00000000e+00+0.j, 0.00000000e+00+0.j, 1.00000000e+00+0.j, ...,
        0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j],
       ...,
       [0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, ...,
                   inf+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j],
       [0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, ...,
        0.00000000e+00+0.j,            inf+0.j, 0.00000000e+00+0.j],
       [0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, ...,
        0.00000000e+00+0.j, 0.00000000e+00+0.j,            inf+0.j]])

In [40]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.algorithms.optimizers import COBYLA
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit.algorithms.minimum_eigensolvers import SamplingVQE
from qiskit import execute, Aer
from qiskit.primitives import Sampler

# set classical optimizer
optimizer = COBYLA(maxiter=50)

# set variational ansatz
ansatz = RealAmplitudes(reps=1)

counts = []
values = []


def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)


# initialize VQE using CVaR with alpha = 0.1
vqe = SamplingVQE(
    Sampler(),
    ansatz=ansatz,
    optimizer=optimizer,
    aggregation=0.1,
    callback=store_intermediate_result,
)

raw_result = vqe.compute_minimum_eigenvalue(qubit_op)
print(raw_result)

SamplingMinimumEigensolverResult:
	Eigenvalue: -1.0190000000001191
	Best measurement
: {'state': 37, 'bitstring': '100101', 'value': (-1.0190000000001191+0j), 'probability': 0.3103856826285104}



In [41]:
from qiskit import QuantumCircuit
from qiskit.algorithms import IterativePhaseEstimation

In [42]:
num_iterations = 3
quantum_instance = None
qc = QuantumCircuit()
sampler = Sampler()
ipe = IterativePhaseEstimation(num_iterations, quantum_instance, sampler)

In [44]:
import numpy as np
m = qubit_op_matrix
np.allclose(np.eye(len(m)), m.dot(m.T.conj()))

False

In [45]:
m = qubit_op_unitary
np.allclose(np.eye(len(m)), m.dot(m.T.conj()))

False

In [46]:
#qc_qubit_op = QuantumCircuit(3)
#qc_qubit_op.unitary(qubit_op_unitary, [0,1,2])
#print(type(qc_qubit_op))

ExtensionError: 'Input matrix is not unitary.'

In [ ]:
result = ipe.estimate(qc_qubit_op, ansatz)
result